import sys
sys.executable

In [2]:
import librosa
import os,glob,pickle
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import soundfile as sf

In [3]:
from audiomentations import Compose, AddGaussianNoise, PitchShift, HighPassFilter

In [4]:
def featureExtractor(audio):
    librosa_audio_data, librosa_sample_rate=librosa.load(audio)
    mfccs_features= librosa.feature.mfcc(y=librosa_audio_data, sr= librosa_sample_rate,n_mfcc=50)
#     print(librosa_sample_rate)
    return mfccs_features

In [5]:
def featureExtractor2(audio,sr):
    mfccs_features= librosa.feature.mfcc(y=audio, sr= sr,n_mfcc=50)
    return mfccs_features

In [6]:
x=[]
y=[]
def load_data():
    for file in glob.glob(r"D:\BTP\infant cry\donateacry_corpus_cleaned_and_updated_data\*"):
        label = os.path.basename(file)
        for audio in glob.glob(file+"\*.wav"):
            x.append(audio)
            y.append(label)


In [7]:
load_data()
print(len(x),len(y))

457 457


In [8]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
print(len(X_train),len(y_train),len(X_test),len(y_test))

365 365 92 92


In [9]:
# def add_noise(signal, noise_percentage_factor):
#     noise = np.random.normal(0, signal.std(), signal.size)
#     augmented_signal = signal + noise * noise_percentage_factor
#     return augmented_signal

In [10]:
def shift_time(data, sampling_rate, shift_max, shift_direction):
    shift = np.random.randint(sampling_rate * shift_max)
    if shift_direction == 'right':
        shift = -shift
    elif shift_direction == 'both':
        direction = np.random.randint(0, 2)
        if direction == 1:
            shift = -shift
    augmented_data = np.roll(data, shift)
    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
    return augmented_data

In [11]:
def time_stretch(signal, time_stretch_rate):
    return librosa.effects.time_stretch(signal, time_stretch_rate)


In [12]:
def pitch_scale(signal, sr, num_semitones):
    return librosa.effects.pitch_shift(signal, sr, num_semitones)

In [13]:
import random
def random_gain(signal, min_factor=0.1, max_factor=0.12):
    gain_rate = random.uniform(min_factor, max_factor)
    augmented_signal = signal * gain_rate
    return augmented_signal

In [14]:
# def invert_polarity(signal):
#     return signal * -1

In [15]:
# Raw audio augmentation
augment_raw_audio = Compose(
    [
        AddGaussianNoise(min_amplitude=0.01, max_amplitude=0.2, p=1),
        PitchShift(min_semitones=-8, max_semitones=8, p=1),
        HighPassFilter(min_cutoff_freq=2000, max_cutoff_freq=4000, p=1)
    ]
)


In [16]:
aug_data=[]
aug_label=[]
i=0
for audio,label in zip(X_train,y_train):
    if label != 'hungry':
        signal,sr=librosa.load(audio)
        augmented_signal1=shift_time(signal,sr,2,'both')
        augmented_signal2=time_stretch(signal,0.5)
        augmented_signal3=pitch_scale(signal,sr,2)
        augmented_signal4=shift_time(signal,sr,3,'right')
        augmented_signal5=random_gain(signal,2,4)
        augmented_signal6=augment_raw_audio(signal,sr)
        
        aug_data.append((augmented_signal1,sr))
        aug_label.append(label)
        aug_data.append((augmented_signal2,sr))
        aug_label.append(label)
        aug_data.append((augmented_signal3,sr))
        aug_label.append(label)
        aug_data.append((augmented_signal4,sr))
        aug_label.append(label)
        aug_data.append((augmented_signal5,sr))
        aug_label.append(label)
        aug_data.append((augmented_signal6,sr))
        aug_label.append(label)
#         sf.write("augmented_signal1.wav", augmented_signal, sr)


In [17]:
print(len(aug_data),len(aug_label))

372 372


In [18]:
extracted_features=[]

def extractFeatures(X,Y):
    x=[]
    y=[]
    for audio,label in zip(X,Y):
        signal,sr=librosa.load(audio)
        feature = featureExtractor2(signal,sr)
        ar=np.resize(feature,(50,280))
        x.append(ar)
        y.append(label)
        extracted_features.append([feature,label])
    return x,y

In [19]:
x_train,y_train=extractFeatures(X_train,y_train)

In [20]:
x_test,y_test=extractFeatures(X_test,y_test)

In [21]:
def extractFeatures2(X,Y):
    x=[]
    y=[]
    for audio,label in zip(X,Y):
        singal=audio[0]
        sr=audio[1]
        feature = featureExtractor2(signal,sr)
        ar=np.resize(feature,(50,280))
        x.append(ar)
        y.append(label)
        extracted_features.append([feature,label])
    return x,y

In [22]:
augX,augY=extractFeatures2(aug_data,aug_label)

In [23]:
print(type(x_train),type(augX))
print(type(x_train[0]),type(augX[0]))
print(x_train[0].shape,augX[0].shape)
print(type(y_train),type(augY))
print(type(y_train[0]),type(augY[0]))

<class 'list'> <class 'list'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(50, 280) (50, 280)
<class 'list'> <class 'list'>
<class 'str'> <class 'str'>


In [24]:
for feature,label in zip(augX,augY):
    x_train.append(feature)
    y_train.append(label)

In [28]:
# print(type(x_train))
# print(type(y_train))
# print(len(x_train))
# print(x_train[0].shape)

In [29]:
length=len(x_train)
x_train=np.array(x_train).reshape(length,50,280)
y_train=np.array(y_train)
print(x_train.shape)
print(y_train.shape)

(737, 50, 280)
(737,)


In [30]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y_train=to_categorical(labelencoder.fit_transform(y_train))
y_train.shape

(737, 5)

In [31]:
# print(len(x_test))
# print(x_test[0].shape)
# print(len(y_test))

In [32]:
x_test=np.array(x_test).reshape(92,50,280)
y_test=np.array(y_test)
print(x_test.shape)
print(y_test.shape)

(92, 50, 280)
(92,)


In [33]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y_test=to_categorical(labelencoder.fit_transform(y_test))
y_test.shape

(92, 5)

In [34]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)
print(x_train.shape,x_test.shape)

(737, 50, 280, 1) (92, 50, 280, 1)


In [35]:
# BUILDING MODEL

import tensorflow as tf
# print(tf.__version__)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow import keras as keras
from sklearn import metrics

# num_labels=y.shape[1]

def build_model(input_shape):
    model=Sequential()
    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu',input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3,3), strides=(2,2),padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
    model.add(keras.layers.MaxPool2D((3,3), strides=(2,2),padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2,2), activation='relu'))
    model.add(keras.layers.MaxPool2D((2,2), strides=(2,2),padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten into 1D array
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64,activation='relu'))
    model.add(keras.layers.Dropout(0.5))

    # output layer
    model.add(keras.layers.Dense(5))
    model.add(Activation('softmax'))
    return model

input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])
model=build_model(input_shape)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 278, 32)       320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 139, 32)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 24, 139, 32)       128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 137, 32)       9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 69, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 69, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 68, 32)        4

In [43]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [44]:
# Training
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

optimizer=keras.optimizers.Adam(learning_rate=0.001)
# model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

start=datetime.now()
# history = model.fit(x_train, y_train, validation_split=0.3, epochs=10, verbose=0)
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=32,epochs=15)

duration=datetime.now() - start
print("Training completed in time: ",duration)


Epoch 1/15
24/24 [==============================] - 10s 334ms/step - loss: 0.3902 - acc: 0.6311 - f1_m: 0.5888 - precision_m: 0.9066 - recall_m: 0.4414 - val_loss: 0.3128 - val_acc: 0.8261 - val_f1_m: 0.8292 - val_precision_m: 0.8341 - val_recall_m: 0.8244
Epoch 2/15
24/24 [==============================] - 8s 316ms/step - loss: 0.3362 - acc: 0.6906 - f1_m: 0.6275 - precision_m: 0.7851 - recall_m: 0.5291 - val_loss: 0.4380 - val_acc: 0.8587 - val_f1_m: 0.8586 - val_precision_m: 0.8586 - val_recall_m: 0.8586
Epoch 3/15
24/24 [==============================] - 7s 300ms/step - loss: 0.3372 - acc: 0.6521 - f1_m: 0.6346 - precision_m: 0.8594 - recall_m: 0.5144 - val_loss: 0.3350 - val_acc: 0.8478 - val_f1_m: 0.8517 - val_precision_m: 0.8569 - val_recall_m: 0.8467
Epoch 4/15
24/24 [==============================] - 8s 316ms/step - loss: 0.2991 - acc: 0.6916 - f1_m: 0.6400 - precision_m: 0.8119 - recall_m: 0.5353 - val_loss: 0.4089 - val_acc: 0.8587 - val_f1_m: 0.8586 - val_precision_m: 0.858

In [47]:
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)
# test_accuracy=model.evaluate(x_test,y_test,verbose=1)
# print(test_accuracy[1])
# test_accuracy
print("loss: ",loss)
print("f1 score: ",f1_score)
print("precision: ",precision)
print("accuracy: ",accuracy)
print("recall: ",recall)

loss:  0.5432752966880798
f1 score:  0.8482142090797424
precision:  0.848214328289032
accuracy:  0.8478260636329651
recall:  0.848214328289032


In [48]:
y_pred = model.predict(x_test) 
y_pred = np.argmax(y_pred, axis = 1) 
label = np.argmax(y_test,axis = 1) 

# print(pred.shape)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(label,y_pred))
# print("y_test:\n", label)
# print("y_pred:\n",y_pred) 


[[ 0  0  0  5  0]
 [ 0  0  0  1  0]
 [ 0  0  0  1  0]
 [ 0  0  1 78  0]
 [ 0  0  0  6  0]]
